# Okamoto-Tanaka Revisited protocol


In [1]:
import random
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
import math

KEY_SIZE = 512
H1_EXPONENT_SIZE = 256
SESSION_KEY_SIZE = 128
EPHEMERAL_EXPONENT_SIZE = 256
ID_SIZE = 256
ENDIAN = "little"


In [2]:
#dobór funkcji haschującej
def select_SHA_context(length_in_bits):
    if length_in_bits <= 224:
        return hashes.SHA224()
    elif length_in_bits <= 256:
        return hashes.SHA256()
    elif length_in_bits <= 384:
        return hashes.SHA384()
    elif length_in_bits <= 512:
        return hashes.SHA512()
    else:
        #TODO Exception
        print("Error, output to big for now")

#hf_KGC = select_SHA_context(H1_EXPONENT_SIZE)
#hf_USER = select_SHA_context(SESSION_KEY_SIZE)


In [3]:
def eea(_a,_b):
    u1 = 1
    v1 = 0
    u2 = 0
    v2 = 1
    if _a < 1 or _b < 1:
        return "Error"
    
    if _b>_a:
        a = _b
        b = _a
    else:
        a = _a
        b = _b
    r = _b
    while r!=0:
        g = r
        q = a//b
        
        r = a - (q*b)   
        u = u1 - (q*u2)
        v = v1 - (q*v2)
        u1 = u2
        v1 = v2
        u2 = u
        v2 = v
        
        a = b
        b = r
        
        #print(q,r,u,v)
    return (a, u1, v1)

#print(eea(77, 11))
#print(eea(62, 7))

## Generacja klucza

### Generacja klucza RSA dla KGC:

In [4]:
class KGC_key_public:
    def __init__(self, N, e, QRN_generator, h1sf, h2f, key_size):
        self.N = N
        self.e = e
        self.QRN_generator = QRN_generator
        self.h1sf = h1sf
        self.h2f = h2f
        self.key_size = key_size

class KGC_key:
    def __init__(self, h1sf, h2f,key_size = KEY_SIZE):
        
        rsa_key = rsa.generate_private_key(
            public_exponent=65537,
            key_size = key_size,
            backend = default_backend()
        )
        self.key_size = key_size
        self.N = rsa_key.public_key().public_numbers().n
        self.e = rsa_key.public_key().public_numbers().e
        self.d = rsa_key.private_numbers().d
        p = rsa_key.private_numbers().p
        q = rsa_key.private_numbers().q
        self.Phi = (p-1)*(q-1)
        self.h1sf = h1sf # część funkcji hashującej H (wynik całej H musi być w QRN)
        self.h2f = h2f #druga funkcja hashująca jest w całości
    def set_QRN_generator(self, generator):
        self.QRN_generator = generator
    def public(self):
        #return (self.N,self.e,self.generator, self.h1sf, self.h2f)
        return KGC_key_public(self.N, self.e, self.QRN_generator, self.h1sf, self.h2f, self.key_size)
    # d,Phi - traktować jak prywatne

hash1_subfunction = select_SHA_context(H1_EXPONENT_SIZE)
hash2_function = select_SHA_context(SESSION_KEY_SIZE)
    
kgc01 = KGC_key(hash1_subfunction, hash2_function)
        

In [5]:
def hashH1(material, kgc_public, endian = ENDIAN):
    context = hashes.Hash(kgc_public.h1sf, backend=default_backend()) 
    context.update(material)
    exponent = int.from_bytes(context.finalize(), byteorder = endian)
    hsh = pow(kgc_public.QRN_generator, exponent, kgc_public.N)
    return hsh


### Generacja kluczy uczestników mOT
#### Znalezienie generatora g grupy reszt kwadratowych mod N
Chcemy znaleźć generator podgrupy reszt kwadratowych w grupue Z_N. Oczekujemy, że grupa reszt będzie odpowiednio duża. Podgrupę będziemy oznaczać __QRN__ (analogicznie do oznaczenia w pracy Gannaro-Krawczyka-Rabina), generator będzie oznaczany __g__.
___
##### Rozkład Phi = d * (2^r) 
Najpierw wyznaczamy rozkład wartości funkcji Eulera dla _modułu klucza RSA_ __KGC__. Chcemy znaleźć takie __t__ oraz __r__, że __Phi = t * (2^r)__, przy czym __t__ jest nieparzyste.

In [6]:
#Rozłożenie N-1 = d* (2**r)
def mr(number):
    t = number
    r = 0
    while t%2 == 0:
    #while math.fmod(d,2) ==0:
        t//=2
        r+=1
    return (t,r)

(t,r) = mr(kgc01.Phi)
print(t,r)
print(t*(2**r))


1303112306350906080995099017537534867234216748228433335406258385015439074423732834087369450263757054486708349072567162278730650978565549944457529052229207 3
10424898450807248647960792140300278937873733985827466683250067080123512595389862672698955602110056435893666792580537298229845207828524399555660232417833656


##### Wyznaczenie generatora g grupy QR_N
W celu wygenerowania akceptowalnego g losujemy pewnwe __alpha__ i sprawdzamy czy generuje podgrupę o satysfakcjonująco dużym rzędzie. Załóżmy tutaj, że zadowala nas jeżeli __alpha__ podniesione do potęgi t modulo N jest różne od 1.
W dalszej części podnosilibyśmy do kwadratu maksymalnie r razy. Jeżeli __alpha__ nie spełnia oczekiwań losujemy je ponownie i powtarzamy procedurę.  
_W przyszłości rozważyć szukanie większych podgrup (podnieść kilka razy do kwadratu i sprawdzić po ilu podniesieniach jest 1)._

In [7]:
#generator grupy Z_N: Znaleźć alpha że g = alpha**2 jest generatorem dużej podgrupy (reszt kwadratowych)
cont = True
while cont == True:
    alpha = random.randint(1,kgc01.N)
    #print (pow(alpha, Phi, N))
    beta = pow(alpha,t,kgc01.N)
    print (beta, alpha)
    if  beta != 1:
        cont = False
        #print ("alpha jest generatorem dużej podgrupy")
    
g = pow(alpha,2,kgc01.N)
kgc01.set_QRN_generator(g)
print (pow(kgc01.QRN_generator, kgc01.Phi, kgc01.N))

1709636145972172425804294904041115825781240473913523312080817708965958578577930757887576306462867632297083810508149849688853287266110031233867033568527139 2396310253904500331824879898349744013996109602522791855327894146579745157271861513904768579542193536691871227969342184664391252880292577120830846680235779
1


#### Klasa użytkowników

In [8]:
#Użytkownik
class User:
    def __init__(self,name,user_id):
        self.name = name
        self.id = user_id.to_bytes(ID_SIZE//8, byteorder=ENDIAN)
        self.idn = int.from_bytes(self.id, byteorder=ENDIAN)
        
    def set_private_key(self,key, endian = ENDIAN):
        self.private_key = key
        self.private_key_n = int.from_bytes(key, endian)
    def generate_ephemeral_exponent(self, ephemeral_exponent_size = EPHEMERAL_EXPONENT_SIZE):
        self.ephemeral_exponent = random.randint(2**(ephemeral_exponent_size//4),2**(ephemeral_exponent_size))
    def calculate_shared_secred(self, msg, id_corr, kgc_public, endian = ENDIAN):
        #print("id_corr", id_corr)
        corr_idn = int.from_bytes(id_corr, endian)
        hsh = hashH1(corr_idn.to_bytes(kgc_public.key_size//8, endian), kgc_public)
        rev_hsh = eea(kgc_public.N, hsh)[2]
        
        print("with reversing hsh:", hsh*rev_hsh%kgc_public.N)
        
        
        
        msgn = int.from_bytes(msg,endian)
        msgn_e = pow(msgn, kgc_public.e, kgc_public.N)
        
        t = (msgn_e * rev_hsh)%kgc_public.N
        K = pow(t, 2*self.ephemeral_exponent, kgc_public.N)
        
        return K
        
        
    
user_A = User("A",random.randint(1,2**ID_SIZE))
user_B = User("B",random.randint(1,2**ID_SIZE))

#### Generacja kluczy prywatnych uczestników mOT.

In [9]:
# funkcja hashująca do grupy reszt kwadratowych
def generate_user_ltk(kgc, material, length_in_bits = H1_EXPONENT_SIZE, key_size = KEY_SIZE, endian = ENDIAN):
    #context = hashes.Hash(kgc.h1sf, backend=default_backend()) 
    #context.update(material)
    #exponent = int.from_bytes(context.finalize(), byteorder = endian)
    #sk = pow(kgc.QRN_generator, exponent, kgc01.N)
    hsh = hashH1(material, kgc.public(), endian)
    return pow(hsh, kgc.d, kgc.N).to_bytes(key_size//8, endian)

user_A.set_private_key(generate_user_ltk(kgc01, user_A.id))
user_B.set_private_key(generate_user_ltk(kgc01, user_B.id))
print(user_A.private_key)
    

    

b'WL\x8bKV>\xa9/|\xb3r\xe4\x93\x81j\x95#\xce-\xe4\xbeW\xda\x0f\x7f\x80S\xfc\xf5\x9f\xcbM\x00\r\xe3\x11\xba\xa3(\xef\xf0\x81\xe0Hn\xb3\x12$\xf5\x83*\x88\xef\xe8\xe2\xeb\x14\x11i\x99\xa4;\x89e'


##### Weryfikacja klucza 

In [10]:
usk = int.from_bytes(user_A.private_key, byteorder = ENDIAN)
usk_pow_e = pow(usk,kgc01.e,kgc01.N)
hash_context = hashes.Hash(select_SHA_context(H1_EXPONENT_SIZE), backend=default_backend()) 
hash_context.update(user_A.id)
exponent = int.from_bytes(hash_context.finalize(), byteorder = ENDIAN)
sk_pre_d = pow(kgc01.QRN_generator, exponent, kgc01.N)
print ( sk_pre_d == usk_pow_e)
if(sk_pre_d != usk_pow_e):
    print (usk_pow_e, sk_pre_d )



True


## Przebieg sesji

### Generacja wiadomości


In [11]:

def generate_negotiation_message(user, kgc_public, endian = ENDIAN):
    g_p = pow(kgc_public.QRN_generator, user.ephemeral_exponent, kgc_public.N)
    return ((g_p*user.private_key_n)%kgc_public.N).to_bytes(kgc_public.key_size,byteorder = endian)
    

user_A.generate_ephemeral_exponent()
user_B.generate_ephemeral_exponent()

msg_A = generate_negotiation_message(user_A,kgc01.public())
msg_B = generate_negotiation_message(user_B,kgc01.public())

#print(msg_A)

    

#idA = bytes(user_A.id.to_bytes(ID_SIZE, byteorder="little"))
#idA = bytes(user_A.id.to_bytes(ID_SIZE, byteorder="big"))
print(user_A.id)



#print(idA)
#print(hex(user_A.id))

b'\xa0\xe1\xd1\x04\xca\x8c\x1a3H;\x079\xed\xe0\xe9\x00\xd9\xc1I\xf4He\xdc\xf3\xd3\xc5jH\x8a\x96\x96h'


In [12]:
k_A = user_A.calculate_shared_secred(msg_B, user_B.id, kgc01.public())
k_B = user_B.calculate_shared_secred(msg_A, user_A.id, kgc01.public())

k = pow(kgc01.QRN_generator,2*user_A.ephemeral_exponent*user_B.ephemeral_exponent*kgc01.e, kgc01.N)
#print (kgc01.QRN_generator)
print("k=\t", k)
print("k_A=\t", k_A)
print("k_B=\t", k_B)

k1a = pow(int.from_bytes(msg_B, byteorder = ENDIAN), kgc01.e, kgc01.N)
k2a = hashH1(user_B.id, kgc01.public())
k3a = k1a*k2a%kgc01.N
k4a = pow(k3a, 2*user_A.ephemeral_exponent, kgc01.N)
print("k1a =\t", k1a)
print("k2a =\t", k2a)
print("k3a =\t", k3a)
print("k4a =\t", k4a)



with reversing hsh: 1
with reversing hsh: 1
k=	 6558190767166387817552187434124154347942563104944066544505805087164637740284194931037169838881126951878207332322618367099976314661323488968411025712367546
k_A=	 7741911533651434925119895011939931347574083655790909081057426663596166558836842910262202075216889149773126871558832122577147358846181126821838972658200244
k_B=	 8001621366603344785316829682201491007139016122555732305396978861940835063340663724317639397259193538477063979834211707310042739729826297452983500216786418
k1a =	 9544449005171428681859333521079880554049739685270007309477227800540304079363373221293134917218690950451346448384641272354216632311108835644490869147483962
k2a =	 8560025960675328523022581531845149622769683291052947753011008209770246264878740393153434740614611523054539735463278955541618533275577405606036525705455351
k3a =	 426711179709150080189048541594009751012501577454241752535675036682908900144397358402308990030058192053363491668298172307626369135797370524781670

In [13]:



rid_A = eea(kgc01.N,user_A.idn)[2]
print((rid_A * user_A.idn)%kgc01.N)

1


In [14]:
#TEST
digest1 = hashes.Hash(hashes.SHA256(), backend=default_backend())
digest2 = hashes.Hash(hashes.SHA256(), backend=default_backend())
digest3 = hashes.Hash(hashes.SHA256(), backend=default_backend())
digest1.update(b"abc")
digest1.update(b"123")
digest2.update(b"123")
digest3.update(b"abc123")

print(digest1.finalize())
print(digest2.finalize())
print(digest3.finalize())

print("1:\t", digest1)
print("2:\t", digest2)
print("3:\t", digest3)
    #TODO

#Generacja klucza użytkownika A:

#user_A.set_private_key(pk)
#Generacja klucza użytkownika B:

#user_B.set_private_key(pk)

b'l\xa1=R\xcap\xc8\x83\xe0\xf0\xbb\x10\x1eBZ\x89\xe8bM\xe5\x1d\xb2\xd29%\x93\xafj\x84\x11\x80\x90'
b'\xa6e\xa4Y B/\x9dA~Hg\xef\xdcO\xb8\xa0J\x1f?\xff\x1f\xa0~\x99\x8e\x86\xf7\xf7\xa2z\xe3'
b'l\xa1=R\xcap\xc8\x83\xe0\xf0\xbb\x10\x1eBZ\x89\xe8bM\xe5\x1d\xb2\xd29%\x93\xafj\x84\x11\x80\x90'
1:	 <cryptography.hazmat.primitives.hashes.Hash object at 0x0000017D3B1240F0>
2:	 <cryptography.hazmat.primitives.hashes.Hash object at 0x0000017D3B124128>
3:	 <cryptography.hazmat.primitives.hashes.Hash object at 0x0000017D3B124240>
